# Part 4: Conservative min–max monthly CVaR (2019)

_Replace this with your work._

In [ ]:
# Your code starts here
# Part 4: Minimize the maximum monthly CVaR in 2019
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog

BETA = 0.95
R_TARGET = 0.0002
INDEX_COL = "NDX"

def _preferred_path(fname: str) -> str:
    if os.path.exists(fname): return fname
    alt = os.path.join("/mnt/data", fname)
    return alt if os.path.exists(alt) else fname

prices_2019_path = _preferred_path("stocks2019.csv")

# Robust datetime indexer
def ensure_datetime_index(df: pd.DataFrame) -> pd.DataFrame:
    cand_names = {"date","timestamp","time","day","tradingday"}
    date_col = None
    for c in df.columns:
        if c.lower() in cand_names:
            date_col = c
            break
    if date_col is None:
        first = df.columns[0]
        try:
            _ = pd.to_datetime(df[first])
            date_col = first
        except Exception:
            raise ValueError(
                "Could not find/parse a date column. "
                f"Available columns: {list(df.columns)}. "
                "Rename your date column to 'Date' or ensure the first column is a parseable date."
            )
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors="coerce")
    out = out.dropna(subset=[date_col]).sort_values(date_col).set_index(date_col)
    return out

def returns_from_prices(prices: pd.DataFrame) -> pd.DataFrame:
    prices = ensure_datetime_index(prices)
    prices = prices.select_dtypes(include=[np.number])
    return prices.pct_change().dropna(how="any")

def monthly_groups(df: pd.DataFrame):
    out = {}
    for (y,m), g in df.groupby([df.index.year, df.index.month]):
        out[f"{int(y):04d}-{int(m):02d}"] = g
    return dict(sorted(out.items()))

def empirical_cvar(losses: np.ndarray, beta: float) -> float:
    var = np.quantile(losses, beta, method="higher")
    tail = losses[losses >= var]
    return float(tail.mean()) if tail.size else float(var)

prices_2019 = pd.read_csv(prices_2019_path)
rets19_all = returns_from_prices(prices_2019)
if INDEX_COL not in rets19_all.columns:
    raise ValueError(f"Expected index column '{INDEX_COL}' in CSV. Got: {list(rets19_all.columns)[:5]}...")

assets = [c for c in rets19_all.columns if c != INDEX_COL]
R19 = rets19_all[assets]
mu = R19.mean(0).values
n = R19.shape[1]

groups = monthly_groups(R19)
months = list(groups.keys())
q_per_m = [groups[m].shape[0] for m in months]
M = len(months)

# Decision order: [x(n), t(1), alpha(M), u blocks]
idx_x0, idx_x1 = 0, n
idx_t = idx_x1
idx_alpha0 = idx_t + 1
idx_u0 = idx_alpha0 + M
N = n + 1 + M + sum(q_per_m)

c = np.zeros(N); c[idx_t] = 1.0  # minimize t
bounds = [(0,None)]*n + [(None,None)] + [(None,None)]*M
for qm in q_per_m: bounds += [(0,None)]*qm

A_ub, b_ub = [], []
u_ptr = idx_u0
for mi, m in enumerate(months):
    Ym = groups[m].values; qm = Ym.shape[0]
    for k in range(qm):
        row = np.zeros(N)
        row[idx_x0:idx_x1] = -Ym[k,:]
        row[idx_alpha0+mi] = -1.0
        row[u_ptr+k] = -1.0
        A_ub.append(row); b_ub.append(0.0)
    row = np.zeros(N)
    row[idx_alpha0+mi] = 1.0
    row[u_ptr:u_ptr+qm] = 1.0/((1.0-BETA)*qm)
    row[idx_t] = -1.0
    A_ub.append(row); b_ub.append(0.0)
    u_ptr += qm

row = np.zeros(N); row[idx_x0:idx_x1] = -mu
A_ub.append(row); b_ub.append(-R_TARGET)

A_ub = np.vstack(A_ub); b_ub = np.array(b_ub)
A_eq = np.zeros((1,N)); A_eq[0, idx_x0:idx_x1] = 1.0; b_eq = np.array([1.0])

res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
              bounds=bounds, method="highs")
if not res.success: raise RuntimeError(f"LP failed: {res.message}")

z = res.x
x_opt = z[idx_x0:idx_x1]
t_star = z[idx_t]

per_month_cvar = {}
for m in months:
    losses = -(groups[m].values @ x_opt)
    per_month_cvar[m] = empirical_cvar(losses, BETA)

print(f"Part 4 — β={BETA}, R≥{R_TARGET:.6f}")
print(f"Optimal t* (minimized max monthly CVaR, 2019): {t_star:.6f}")

w = pd.Series(x_opt, index=assets, name="weight").sort_values(ascending=False)
display(w)

pm = pd.Series(per_month_cvar, name="CVaR_month")
display(pm)

ax = w.plot(kind="bar", figsize=(13,4), title="Part 4 — Min–Max Monthly CVaR Weights (2019)")
ax.set_ylabel("Weight"); plt.tight_layout(); plt.show()

ax = pm.plot(kind="bar", figsize=(13,4), title="Part 4 — Realized Monthly CVaR (2019) with x*")
ax.set_ylabel("CVaR"); plt.tight_layout(); plt.show()

